# Setup

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import urllib.request

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train.head()
train['Dates'] = pd.to_datetime(train['Dates'])
train.duplicated().sum()
train = train.drop_duplicates()

train = train.drop(columns=["Descript", "Resolution", "Address"])
train.shape

# EDA (Exploratory Data Analysis) & Visualisation


In [ ]:
train['X'].plot.box()

In [ ]:
train['Y'].plot.box()

On voit que les diagrammes moustaches sont cohérents dans l'ensemble, les outliers pour X sont justifier par rapport à la longitude de la ville

Nous allons vérifier les catégories utilisés lors de la déclaration d'incident, en mettant à part celles qui n'ont pas de catégories spécifiées

In [ ]:
train = train[train['Category'] != 'OTHER OFFENSES']
data = train.groupby('Category').count().iloc[:, 0].sort_values(
    ascending=False)
px.bar(data, text_auto='.2s').show()


Nous allons regarder si ces incidents sont liés au district dans lesquels ils ont lieux
Vérifions que les incidents est lieux soit répertorier dans les bons quartier

In [ ]:
px.scatter(train, y="Y", x="X", width = 1100, height = 700, color="PdDistrict").show()


## Impact

### Quartiers et Types d'incident

Nous allons regarder si ces incidents sont liés aux quartiers dans lesquels ils ont lieux


In [ ]:
px.histogram(train, x='PdDistrict', color='Category').show()

In [ ]:
# Test diplay map
# median_X = train['X'].median()
# median_Y = train['Y'].median()
#
# fig = px.scatter_mapbox(
#     train,
#     lat='Y',
#     lon='X',
#     color= "Category",
#     zoom=10,
#     center={"lat": median_Y, "lon": median_X},
#     title="Carte avec arrière-plan"
# )
#
# fig.update_layout(
#     mapbox_style="open-street-map",
#     height=600,
#     margin={"r": 0, "t": 50, "l": 0, "b": 0}
# )
#
#
# fig.show()

On voit que certains quartiers ont un plus haut taux d'incidents et on voit également que la plupart des types d'incidents ont la même répartition pour chaque quartier

#### 1. Journalier
Nous pouvons nous demander si certains jour dans la semaine peuvent impacter certains quartiers


In [ ]:
px.histogram(train, x='DayOfWeek', color='PdDistrict').show()

Il n'y as pas de grande différence au nombre d'incident par jour en fonction des quartiers, sauf pour les vendredi et samedi

Essayons de voir si les catégories sont toutes les mêmes chaque jour

In [ ]:
px.histogram(train, x='DayOfWeek', color='Category').show()

La répartition des types d'incidents en fonction des jours n'a pas non plus de grands impacts

#### 2. Horaires

Il serait alors possile que certains quartiers soient plus impactés que d'autres à une certaine heure

In [ ]:
dhour = train.copy()
dhour["Dates"] = pd.Series(dhour["Dates"]).dt.hour
px.histogram(dhour, x='Dates', color='PdDistrict').show()

Il semble que la proportion d'incident en fonction des heures pour chaque quartiers reste toujours la même

Regardons maintenant pour les types d'incidents

In [ ]:
px.histogram(dhour, x='Dates', color='Category').show()

Nous retrouvons environ la même proportion chaque heure

#### 3. Années

Vérifions si certains quartier sont plus actifs certaines années

In [ ]:
dyear = train.copy()
dyear["Dates"] = pd.Series(dyear["Dates"]).dt.year
px.histogram(dyear, x='Dates', color='PdDistrict').show()

La répartition semble identiques pour chaque district au fil des années, la colonne de 2015 est plus petite car les données s'arrêtent à juin 2015

Regardons également pour les types d'incidents

In [ ]:
px.histogram(dyear, x='Dates', color='Category').show()


Nous voyons que sur les dernières années, il y a une plus grosse proportion de Vol (LARCENY/THEFT)

#### 4. Month

Il se peut que les incidents soient plutôt repartit sur les différents mois des années

In [ ]:
dmonth = train.copy()
dmonth["Dates"] = pd.Series(dmonth["Dates"]).dt.month
px.histogram(dmonth, x='Dates', color='PdDistrict').show()

On voit que les incidents ont une hausse autour de mars, avril, mai, puis à octobre même si l'implications de la plupart des quartiers ne changent pas


In [ ]:
px.histogram(dmonth, x='Dates', color='Category').show()


Pour les types d'incidents, on ne retrouve toujours pas de tendances en fonction du mois au fil des années


In [ ]:
px.pie(train, names='Category').show()

Lorsque nous vérifions la répartition des types d'incidents, on retrouve près d'un quart du temps le vol

Nous pourrons tester les différents modèles sur cette base de données mais aussi sur une plus homogéne en récupérer par exemple 1000 incidents pour les catégories ayant plus de 1000 incidents et en supprimant les incidents apparaissant à titre exceptionnel.

In [ ]:
category_to_reduce = train['Category'].value_counts()[train['Category'].value_counts() > 1000].index

equal_train = (
    train[train['Category'].isin(category_to_reduce)]
    .groupby('Category')
    .apply(lambda x: x.sample(n=1000, random_state=42))
    .reset_index(drop=True)
)

px.pie(equal_train, names='Category').show()